##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX 파이프라인 및 TensorFlow Transform을 사용한 특성 엔지니어링

***TFX 파이프라인을 사용하여 입력 데이터를 변환하고 모델을 훈련합니다.***

참고: 설정이 필요하지 않은 Colab 노트북에서 이 튜토리얼을 실행하는 것이 좋습니다! "Google Colab에서 실행"을 클릭하기만 하면 됩니다.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tft"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/penguin_tft.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/penguin_tft.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/penguin_tft.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>
</td>
</table></div>

이 노트북 기반 튜토리얼에서는 TFX 파이프라인을 생성 및 실행하여 원시 입력 데이터를 수집하고 ML 훈련에 알맞게 전처리합니다. 이 노트북은 [TFX 파이프라인 및 TensorFlow 데이터 검증 튜토리얼을 사용하는 데이터 검증](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv)에서 구축한 TFX 파이프라인을 기반으로 합니다. 해당 내용을 아직 읽지 않았다면 이 노트북을 계속 진행하기 전에 읽어야 합니다.

특성 엔지니어링으로 데이터의 예측 품질을 높이거나 차원을 낮출 수 있습니다. TFX 사용의 이점 중 하나는 변환 코드를 한 번 작성하면 훈련/적용 불일치를 피할 수 있도록 변환 결과가 훈련과 적용 간에 일관성 있게 된다는 것입니다.

파이프라인에 `Transform` 구성 요소를 추가합니다. Transform 구성 요소는 [tf.transform](https://www.tensorflow.org/tfx/transform/get_started) 라이브러리를 사용하여 구현됩니다.

TFX의 다양한 개념에 대해 자세히 알아보려면 [TFX 파이프라인 이해하기](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)를 참조하세요.

## 설정하기

먼저 TFX Python 패키지를 설치하고 모델에 사용할 데이터세트를 다운로드해야 합니다.

### Pip 업그레이드

로컬에서 실행할 때 시스템에서 Pip을 업그레이드하지 않으려면 Colab에서 실행 중인지 확인해야 합니다. 물론 로컬 시스템은 별도로 업그레이드할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX 설치


In [ ]:
!pip install -U tfx

### 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, "런타임 다시 시작" 버튼을 클릭하거나 "런타임 &gt; 런타임 다시 시작..." 메뉴를 사용하여 런타임을 다시 시작해야 합니다. 이는 Colab이 패키지를 로드하는 방식 때문입니다.

TensorFlow 및 TFX 버전을 확인합니다.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 변수 설정하기

파이프라인을 정의하는 데 사용되는 변수가 몇 가지 있습니다. 이러한 변수를 원하는 대로 사용자 정의할 수 있습니다. 기본적으로 파이프라인의 모든 출력은 현재 디렉터리 아래에 생성됩니다.

In [ ]:
import os

PIPELINE_NAME = "penguin-transform"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### 예제 데이터 준비하기

TFX 파이프라인에서 사용할 예제 데이터세트를 다운로드합니다. 사용하는 데이터세트는 [Palmer Penguins 데이터세트](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)입니다.

다만 미리 전처리한 데이터세트를 사용한 이전 튜토리얼과는 달리 이번에는 **원시** Palmer Penguins 데이터세트를 사용합니다.


TFX ExampleGen 구성 요소는 디렉터리로부터 입력을 읽기 때문에 디렉터리를 생성한 후 데이터세트를 디렉터리에 복사해야 합니다.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_path = 'https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_path, _data_filepath)

빠르게 원시 데이터의 형태를 살펴봅니다.

In [ ]:
!head {_data_filepath}

`NA`로 표시되는 누락된 값이 있는 입력 항목이 있습니다. 이 튜토리얼에서는 해당 입력 항목만 삭제합니다.

In [ ]:
!sed -i '/\bNA\b/d' {_data_filepath}
!head {_data_filepath}

펭귄을 묘사하는 7가지 특성을 볼 수 있어야 합니다. 이전 튜토리얼과 동일한 특성 세트('culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g')를 사용하고 펭귄의 '종'을 예측할 것입니다.

**유일한 차이점은 입력 데이터를 전처리하지 않는다는 것입니다.** 이 튜토리얼에서는 '섬' 또는 '성별'과 같은 다른 특성을 사용하지 않습니다.

### 스키마 파일 준비하기

[TFX 파이프라인 및 TensorFlow 데이터 검증을 사용한 데이터 검증](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv)에 설명된 대로 데이터세트용 스키마 파일이 필요합니다. 데이터세트가 이전 튜토리얼과 다르기 때문에 다시 생성해야 합니다. 이 튜토리얼에서는 이러한 단계를 건너뛰고 준비된 스키마 파일만 사용합니다.


In [ ]:
import shutil

SCHEMA_PATH = 'schema'

_schema_uri = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/schema/raw/schema.pbtxt'
_schema_filename = 'schema.pbtxt'
_schema_filepath = os.path.join(SCHEMA_PATH, _schema_filename)

os.makedirs(SCHEMA_PATH, exist_ok=True)
urllib.request.urlretrieve(_schema_uri, _schema_filepath)

이 스키마 파일은 수동 변경한 사항이 없이 이전 튜토리얼과 동일한 파이프라인을 사용하여 생성되었습니다.

## 파이프라인 생성하기

TFX 파이프라인은 Python API를 사용하여 정의합니다. [데이터 검증 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv)에서 생성한 파이프라인에 `Transform` 구성요소를 추가합니다.

변환 구성 요소는 `ExampleGen` 구성 요소의 입력 데이터와 `SchemaGen` 구성 요소의 스키마를 필요로 하며 "변환 그래프"를 생성합니다. 출력 결과는 `Trainer` 구성 요소에서 사용합니다. 변환은 선택적으로 변환 후 구체화된 데이터인 "변환된 데이터"를 추가로 생성할 수 있습니다. 그러나 이 튜토리얼에서는 중간 변환 데이터를 구체화하지 않고 훈련 중에 데이터를 변환합니다.

한 가지 주의할 점은 입력 데이터를 변환하는 방법을 설명하기 위해 Python 함수인 `preprocessing_fn`을 정의해야 한다는 것입니다. 이는 모델 정의를 위해 사용자 코드도 필요로 하는 Trainer 구성 요소와 유사합니다.


### 전처리 및 학습 코드 작성하기

두 개의 Python 함수를 정의해야 합니다. 하나는 Transform이고 다른 하나는 Trainer입니다.

#### preprocessing_fn

Transform 구성 요소는 `Trainer` 구성 요소에서 수행한 작업과 같이 지정한 모듈 파일에서 `preprocessing_fn`이라는 함수를 찾습니다. Transform 구성 요소의 <a href="https://github.com/tensorflow/tfx/blob/142de6e887f26f4101ded7925f60d7d4fe9d42ed/tfx/components/transform/component.py#L113" data-md-type="link">`preprocessing_fn` 매개변수</a>를 사용하여 특정 함수를 지정할 수도 있습니다.

이 예제에서는 두 종류의 변환을 수행합니다. `culmen_length_mm`과 `body_mass_g`와 같은 연속 숫자 특성의 경우 [tft.scale_to_z_score](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score) 함수를 사용하여 이러한 값을 정규화합니다. 레이블 특성의 경우 문자열 레이블을 숫자 인덱스 값으로 변환해야 합니다. 변환에는 [`tf.lookup.StaticHashTable`](https://www.tensorflow.org/api_docs/python/tf/lookup/StaticHashTable)을 사용합니다.

변환한 필드를 쉽게 식별하기 위해 변환한 특성 이름에 `_xf` 접미사를 추가합니다.

#### run_fn

모델 자체는 이전 튜토리얼과 거의 동일하지만 이번에는 Transform 구성 요소의 변환 그래프를 사용하여 입력 데이터를 변환합니다.

이전 튜토리얼과 비교하여 한 가지 더 중요한 차이점은 이제는 모델의 계산 그래프뿐만 아니라 Transform 구성 요소에서 생성한 전처리용 변환 그래프를 포함하는 적용 모델도 내보낸다는 것입니다. 수신하는 요청을 적용하는 데 사용할 별도의 함수를 정의해야 합니다. 훈련 데이터와 적용 요청 모두에 동일한 함수 `_apply_preprocessing`이 사용되었음을 알 수 있습니다.


In [ ]:
_module_file = 'penguin_utils.py'

In [ ]:
%%writefile {_module_file}


from typing import List, Text
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

# Specify features that we will use.
_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10


# NEW: TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature.
  """
  outputs = {}

  # Uses features defined in _FEATURE_KEYS only.
  for key in _FEATURE_KEYS:
    # tft.scale_to_z_score computes the mean and variance of the given feature
    # and scales the output based on the result.
    outputs[key] = tft.scale_to_z_score(inputs[key])

  # For the label column we provide the mapping from string to index.
  # We could instead use `tft.compute_and_apply_vocabulary()` in order to
  # compute the vocabulary dynamically and perform a lookup.
  # Since in this example there are only 3 possible values, we use a hard-coded
  # table for simplicity.
  table_keys = ['Adelie', 'Chinstrap', 'Gentoo']
  initializer = tf.lookup.KeyValueTensorInitializer(
      keys=table_keys,
      values=tf.cast(tf.range(len(table_keys)), tf.int64),
      key_dtype=tf.string,
      value_dtype=tf.int64)
  table = tf.lookup.StaticHashTable(initializer, default_value=-1)
  outputs[_LABEL_KEY] = table.lookup(inputs[_LABEL_KEY])

  return outputs


# NEW: This function will apply the same transform operation to training data
#      and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  transformed_features = tft_layer(raw_features)
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  else:
    return transformed_features, None


# NEW: This function will create a handler function which gets a serialized
#      tf.example, preprocess and run an inference with it.
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and
  # tracked.
  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    # Expected input is a string which is serialized tf.Example format.
    feature_spec = tf_transform_output.raw_feature_spec()
    # Because input schema includes unnecessary fields like 'species' and
    # 'island', we filter feature_spec to include required keys only.
    required_feature_spec = {
        k: v for k, v in feature_spec.items() if k in _FEATURE_KEYS
    }
    parsed_features = tf.io.parse_example(serialized_tf_examples,
                                          required_feature_spec)

    # Preprocess parsed input with transform operation defined in
    # preprocessing_fn().
    transformed_features, _ = _apply_preprocessing(parsed_features,
                                                   model.tft_layer)
    # Run inference with ML model.
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
      schema=tf_transform_output.raw_metadata.schema)

  transform_layer = tf_transform_output.transform_features_layer()
  def apply_transform(raw_features):
    return _apply_preprocessing(raw_features, transform_layer)

  return dataset.map(apply_transform).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [
      keras.layers.Input(shape=(1,), name=key)
      for key in _FEATURE_KEYS
  ]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # NEW: Save a computation graph including transform layer.
  signatures = {
      'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

이제 TFX 파이프라인 구축에 필요한 모든 준비 단계를 완료했습니다.

### 파이프라인 정의 작성하기

TFX 파이프라인을 생성하는 함수를 정의합니다. `Pipeline` 객체는 TFX가 지원하는 파이프라인 오케스트레이션 시스템 중 하나를 사용하여 실행할 수 있는 TFX 파이프라인을 나타냅니다.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # NEW: Transforms input data using preprocessing_fn in the 'module_file'.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      materialize=False,
      module_file=module_file)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],

      # NEW: Pass transform_graph to the trainer.
      transform_graph=transform.outputs['transform_graph'],

      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      statistics_gen,
      schema_importer,
      example_validator,

      transform,  # NEW: Transform component was added to the pipeline.

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## 파이프라인 실행하기

이전 튜토리얼과 같이 `LocalDagRunner`를 사용합니다.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

파이프라인이 성공적으로 완료되면 "INFO:absl:Component Pusher is finished." 메시지가 표시됩니다.

이전 단계에서 변수를 변경하지 않은 경우 푸셔 구성 요소는 훈련한 모델을 `serving_model/penguin-transform` 디렉터리인 `SERVING_MODEL_DIR`로 푸시합니다. Colab의 왼쪽 패널에서 혹은 다음 명령을 사용하여 파일 브라우저에서 결과를 볼 수 있습니다.

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

[`saved_model_cli` 도구](https://www.tensorflow.org/guide/saved_model#show_command)를 사용하여 생성한 모델의 서명을 확인할 수도 있습니다.

In [ ]:
!saved_model_cli show --dir {SERVING_MODEL_DIR}/$(ls -1 {SERVING_MODEL_DIR} | sort -nr | head -1) --tag_set serve --signature_def serving_default

자체 `serve_tf_examples_fn` 함수로 `serving_default`를 정의했기 때문에 서명이 단일 문자열을 사용하는 것으로 표시됩니다. 이 문자열은 tf.Examples의 직렬화된 문자열이며 앞에서 정의했듯이 [tf.io.parse_example()](https://www.tensorflow.org/api_docs/python/tf/io/parse_example) 함수를 사용하여 구문 분석됩니다(tf.Examples에 대한 자세한 내용은 [여기](https://www.tensorflow.org/tutorials/load_data/tfrecord)를 참조).

내보내기를 수행한 모델을 로드하고 몇 개의 예제를 통해 추론을 일부 시도할 수 있습니다.

In [ ]:
# Find a model with the latest timestamp.
model_dirs = (item for item in os.scandir(SERVING_MODEL_DIR) if item.is_dir())
model_path = max(model_dirs, key=lambda i: int(i.name)).path

loaded_model = tf.keras.models.load_model(model_path)
inference_fn = loaded_model.signatures['serving_default']

In [ ]:
# Prepare an example and run inference.
features = {
  'culmen_length_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),
  'culmen_depth_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[16.1])),
  'flipper_length_mm': tf.train.Feature(int64_list=tf.train.Int64List(value=[213])),
  'body_mass_g': tf.train.Feature(int64_list=tf.train.Int64List(value=[5400])),
}
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())

'Gentoo'(젠투 펭귄) 종에 해당하는 세 번째 요소의 값이 셋 중에서 가장 클 것으로 예상됩니다.

## 다음 단계

변환 구성 요소에 대해 자세히 알아보려면 [Transform 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/transform)를 참조하세요. https://www.tensorflow.org/tfx/tutorials에서 더 많은 리소스를 확인할 수 있습니다.

TFX의 다양한 개념에 대해 자세히 알아보려면 [TFX 파이프라인 이해하기](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)를 참조하세요.
